In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB
from sklearn.decomposition import TruncatedSVD
from nltk.corpus import stopwords
import string
import re
import pandas as pd
import time
import os
import numpy as np
import pickle

//anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
//anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [3]:
# DEFINE DIRECTORY PATH
path_to_json = 'TrainingData/'

# CREATE LIST OF FILES FROM THE DIRECTORY
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# DEFINE PANDAS DATAFRAME
df = pd.DataFrame()

# LOOP THROUGH FILES, READ IN JSON AND BUILD DATAFRAME
for index, js in enumerate(json_files):
    json_data = pd.read_json(os.path.join(path_to_json, js), 'r')
    df = df.append(json_data)
    
# LOOK AT TOPIC DICTIONARY AND GET A TOPIC COUNT
topic_file = open('./topicDictionary.txt', 'r')
topics = topic_file.read().split('\r\n')

# SPLITTING THE ELEMENTS OF THE JSON INTO TEXT, PUBLICATION DATE AND TOPICS
df['text'] = df.TrainingData.apply(lambda x: x['bodyText'])
df['pubdate'] = df.TrainingData.apply(lambda x: x['webPublicationDate'])
df['topics'] = df.TrainingData.apply(lambda x: x['topics'])

# DROP FIRST TWO COLUMNS
df.reset_index(inplace=True, drop=True)
df.drop('TrainingData', axis=1, inplace=True)

# DEFINE FUNCTION TO CREATE OUR DATAFRAME
def topic_col(x):
    a = 0
    for elem in x:
        if elem == topic:
            a = 1
    return a

# RUN TOPIC COL FUNCTION ON ALL DATA
for topic in topics:
    time1 = time.time()
    df[topic] = df['topics'].map(topic_col)
    print topic
    time2 = time.time()
    time_in_s = (time2-time1)
    print 'Function takes around %0.3f seconds to run' % (time_in_s)

activism
Function takes around 2.782 seconds to run
afghanistan
Function takes around 1.763 seconds to run
aid
Function takes around 1.217 seconds to run
algerianhostagecrisis
Function takes around 1.367 seconds to run
alqaida
Function takes around 1.213 seconds to run
alshabaab
Function takes around 1.244 seconds to run
antiwar
Function takes around 1.337 seconds to run
arabandmiddleeastprotests
Function takes around 1.425 seconds to run
armstrade
Function takes around 1.309 seconds to run
australianguncontrol
Function takes around 1.339 seconds to run
australiansecurityandcounterterrorism
Function takes around 1.518 seconds to run
bastilledaytruckattack
Function takes around 1.325 seconds to run
belgium
Function takes around 1.117 seconds to run
berlinchristmasmarketattack
Function takes around 1.308 seconds to run
bigdata
Function takes around 1.073 seconds to run
biometrics
Function takes around 1.141 seconds to run
bokoharam
Function takes around 1.158 seconds to run
bostonmaratho

undercoverpoliceandpolicing
Function takes around 1.336 seconds to run
unitednations
Function takes around 1.180 seconds to run
usguncontrol
Function takes around 1.149 seconds to run
values
Function takes around 1.072 seconds to run
warcrimes
Function takes around 1.163 seconds to run
warreporting
Function takes around 1.145 seconds to run
weaponstechnology
Function takes around 1.199 seconds to run
womeninbusiness
Function takes around 1.181 seconds to run
woolwichattack
Function takes around 1.173 seconds to run
worldmigration
Function takes around 1.169 seconds to run
zikavirus
Function takes around 1.141 seconds to run


In [4]:
# CREATE FINAL TEST DF
testdf = pd.read_json('/Users/sudheerpamula/Downloads/TestData.json', 'r')
testdf['text'] = testdf.TestData.apply(lambda x: x['bodyText'])
testdf['pubdate'] = testdf.TestData.apply(lambda x: x['webPublicationDate'])
testdf['topics'] = testdf.TestData.apply(lambda x: x['topics'])
testdf.drop('TestData', axis=1, inplace=True)

In [5]:
def sampler(topic, df):
    
    # SELECT INDICES OF TOPIC ARTICLES
    topicindexes = df[df[topic] == 1].index.tolist()
    
    # FIND COUNT OF TOPIC ARTICLES
    articlecount = len(topicindexes)
    
    if articlecount < 200:
        print articlecount
        # SELECT NON TOPIC INDICES
        nontopicarticlesindexes = df[df[topic] == 0].sample(articlecount*9).index.tolist()
        nonarticlecount = len(nontopicarticlesindexes)
        # CREATE LIST OF COMBINED INDICES
        sampleindex = topicindexes + nontopicarticlesindexes 
        # CREATE NEW DATAFRAME
        X = df.iloc[sampleindex]['text'].reset_index(drop=True)
        y = df.iloc[sampleindex][topic].reset_index(drop=True)
        X = X.append(df.iloc[topicindexes]['text'])
        y = y.append(df.iloc[topicindexes][topic])
        X.reset_index(drop=True, inplace=True)
        y.reset_index(drop=True, inplace=True) 
    else:
        # SELECT NON TOPIC INDICES
        topicindexes = df[df[topic] == 1].sample(200).index.tolist()
        
        nontopicarticlesindexes = df[df[topic] == 0].sample(len(topicindexes)*9).index.tolist()
        nonarticlecount = len(nontopicarticlesindexes)
        # CREATE LIST OF COMBINED INDICES
        sampleindex = topicindexes + nontopicarticlesindexes
        # CREATE NEW DATAFRAME
        X = df.iloc[sampleindex]['text'].reset_index(drop=True)
        y = df.iloc[sampleindex][topic].reset_index(drop=True)
    
    return X, y

In [6]:
submission = pd.DataFrame(index=testdf.index, columns=topics)

In [6]:
# OPEN VOCABULARY DICTIONARY PICKLE AND EDIT
with open('vocab.pickle', 'rb') as handle:
    modelvocab = pickle.load(handle)

In [7]:
parameters = {
    'vectorizer__max_df': (0.15, 0.25),
    'tfidf__norm': ('l2', 'l1'),
    'clf__penalty': ('l1', 'l2'),
}

clf = LogisticRegression(warm_start=True, n_jobs=-1)

In [8]:
topics_0 = ['activism', 'turkeycoupattempt', 'tunisiaattack2015', 'sanbernardinoshooting', 'peaceandreconciliation',
            'parisattacks', 'orlandoterrorattack', 'munichshooting', 'francetrainattack', 'charliehebdoattack',
            'brusselsattacks', 'berlinchristmasmarketattack','bastilledaytruckattack','zikavirus']

In [ ]:
f1_scores = []
grid_search_params = []

time1 = time.time()

for topic in topics:
    if topic in topics_0:
        submission[topic] = 1
        f1_scores.append(0)
        grid_search_params.append("none")
    else:
        X, y = sampler(topic, df)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)
        
        #N-GRAM RANGE OF (1,1)
        vectorizer = CountVectorizer(ngram_range=(1,4), stop_words='english')

        # PIPELINE OF VECTORIZER, TF-IDF TRANSFORMER, STANDARD SCALER, MODEL
        pipe = Pipeline([('vectorizer', vectorizer), 
                 ('tfidf', TfidfTransformer()),
                 ('ss', StandardScaler(with_mean=False)),
                 ('clf', clf)])

        # GRIDSEARCH PIPELINE FIT
        grid_search = GridSearchCV(pipe, parameters, n_jobs=-1, verbose=1)
        grid_search.fit(X_train, y_train)

        # PREDICT
        preds = grid_search.best_estimator_.predict(X_test)
        topicf1score = f1_score(y_test, preds)
        print "F1 score:", topicf1score
        f1_scores.append(topicf1score)
        grid_search_params.append(grid_search.best_params_)

        # FINAL PREDICTIONS
        preds_final = grid_search.best_estimator_.predict(testdf['text'])

        # WRITE TO CSV
        submission[topic] = preds_final
        
    submission.to_csv('submission_logregngram4.csv')
    print topic
    
time2 = time.time()
time_in_s = (time2-time1)
print 'Function takes around %0.3f seconds to run' % (time_in_s)

scoredf = pd.DataFrame({'f1': f1_scores, 'params': grid_search_params}, index=topics)

activism
Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  3.8min finished


F1 score: 0.891089108911
afghanistan
Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  3.5min finished


F1 score: 0.854368932039
aid
58
Fitting 3 folds for each of 8 candidates, totalling 24 fits


...

F1 score: 0.969696969697
firefighters
178
Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  2.8min finished


F1 score: 0.988636363636
forensicscience
Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  2.8min finished


F1 score: 0.814814814815
france
francetrainattack
Fitting 3 folds for each of 8 candidates, totalling 24 fits


# Continuing to run as had to interrupt script

Edit topic list to choose only topics that haven't been run.

In [11]:
topics = ['freedomofspeech',
 'genevaconventions',
 'germany',
 'guncrime',
 'hacking',
 'hashtags',
 'helicoptercrashes',
 'humanitarianresponse',
 'humanrights',
 'humanrightsact',
 'humantrafficking',
 'immigration',
 'india',
 'indonesia',
 'internallydisplacedpeople',
 'internationalcourtofjustice',
 'internationalcriminaljustice',
 'internetsafety',
 'iraq',
 'isis',
 'israel',
 'jordan',
 'jubilee',
 'judiciary',
 'july7',
 'justiceandsecurity',
 'kenya',
 'knifecrime',
 'lebanon',
 'libya',
 'localgovernment',
 'logistics',
 'london',
 'londonriots',
 'malaysia',
 'mali',
 'malware',
 'metropolitanpolice',
 'middleeastpeacetalks',
 'migration',
 'military',
 'ministryofdefence',
 'morocco',
 'mrsa',
 'mumbaiterrorattacks',
 'munichshooting',
 'naturaldisasters',
 'nigeria',
 'nuclearweapons',
 'occupy',
 'organisedcrime',
 'orlandoterrorattack',
 'osamabinladen',
 'paris',
 'parisattacks',
 'peaceandreconciliation',
 'philippines',
 'piracy',
 'planecrashes',
 'police',
 'protest',
 'refugees',
 'religion',
 'retirementage',
 'rio20earthsummit',
 'royalairforce',
 'royalnavy',
 'russia',
 'sanbernardinoshooting',
 'saudiarabia',
 'september11',
 'slavery',
 'somalia',
 'southafrica',
 'southchinasea',
 'stopandsearch',
 'surveillance',
 'sydneysiege',
 'syria',
 'taliban',
 'terrorism',
 'thailand',
 'torture',
 'traincrashes',
 'transport',
 'tunisiaattack2015',
 'turkey',
 'turkeycoupattempt',
 'ukcrime',
 'uksecurity',
 'uksupremecourt',
 'undercoverpoliceandpolicing',
 'unitednations',
 'usguncontrol',
 'values',
 'warcrimes',
 'warreporting',
 'weaponstechnology',
 'womeninbusiness',
 'woolwichattack',
 'worldmigration',
 'zikavirus']

In [13]:
for topic in topics:
    if topic in topics_0:
        submission[topic] = 1
#         f1_scores.append(0)
#         grid_search_params.append("none")
    else:
        X, y = sampler(topic, df)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y)
        
        #N-GRAM RANGE OF (1,1)
        vectorizer = CountVectorizer(ngram_range=(1,4), stop_words='english')

        # PIPELINE OF VECTORIZER, TF-IDF TRANSFORMER, STANDARD SCALER, MODEL
        pipe = Pipeline([('vectorizer', vectorizer), 
                 ('tfidf', TfidfTransformer()),
                 ('ss', StandardScaler(with_mean=False)),
                 ('clf', clf)])

        # GRIDSEARCH PIPELINE FIT
        grid_search = GridSearchCV(pipe, parameters, n_jobs=-1, verbose=1)
        grid_search.fit(X_train, y_train)

        # PREDICT
        preds = grid_search.best_estimator_.predict(X_test)
        topicf1score = f1_score(y_test, preds)
        print "F1 score:", topicf1score
#         f1_scores.append(topicf1score)
#         grid_search_params.append(grid_search.best_params_)

        # FINAL PREDICTIONS
        preds_final = grid_search.best_estimator_.predict(testdf['text'])

        # WRITE TO CSV
        submission[topic] = preds_final
        
    submission.to_csv('submission_logregngram4.csv')
    print topic
    
time2 = time.time()
time_in_s = (time2-time1)
print 'Function takes around %0.3f seconds to run' % (time_in_s)

# scoredf = pd.DataFrame({'f1': f1_scores, 'params': grid_search_params}, index=topics)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  2.4min finished


F1 score: 0.725490196078
freedomofspeech
84
Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  1.1min finished


F1 score: 0.988235294118
genevaconventions
Fitting 3 folds for each of 8 candidates, totalling 24 fits


...

F1 score: 0.977528089888
womeninbusiness
141
Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  1.9min finished


F1 score: 1.0
woolwichattack
Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  2.5min finished


F1 score: 0.909090909091
worldmigration
zikavirus
Function takes around 21592.568 seconds to run


Scored - .13416
Our score went down after adding n_grams (1,4) 